In [2]:
import numpy as np

## Base selection

In [3]:
def select_bases_random(targets):
    num_targets = len(targets)
    base_indices = np.random.randint(0, num_targets, num_targets)
    return targets[base_indices]

def select_bases_permute(targets):
    return np.random.permutation(targets)

def select_bases_offset(targets, n=5):
    return np.roll(targets, 5)

## Differential generation

In [4]:
def differential_random(choices, F):
    a, b = choices[np.random.randint(0, len(choices), 2)]
    #a, b = choices[np.random.choice(choices.shape[0], 2, replace=False)]
    return F * (a - b)

def differential_random_2(choices, F):
    a, b, c, d = choices[np.random.choice(choices.shape[0], size=4, replace=False)]
    return F * (a - b) + F * (c - d)

## Crossover

In [5]:
def crossover_binomial(target, donor, Cr):
    dim = len(target)
    r = np.random.randint(0, dim)
    p = np.random.rand(dim) > Cr
    p[r] = True
    return np.where(p, target, donor)

## Initialization

In [6]:
def init_uniform_random(n, min_values, max_values):
    d = len(min_values)
    result = np.zeros((n, d))
    for i in range(n):
        result[i, :] = np.random.uniform(min_values, max_values)
    return result
    #why does this not work? docs say output of lambda determines shape
    #return np.fromfunction(lambda num,: np.random.uniform(min_values, max_values), shape = (n,), dtype=int)

## DE main loop

In [19]:
def de(targets, fitness, F, Cr):
    next_population = np.copy(targets)
    bases = select_bases_permute(targets)
    for i in range(len(targets)):#, target, base in enumerate(zip(targets, bases)):
        target = targets[i]
        base = bases[i]
        diff = differential_random(targets, F)
        donor = base + diff
        trial = crossover_binomial(target, donor, Cr)
        if fitness(trial) > fitness(target):
            next_population[i, :] = trial  
    return next_population

## Fitness function

In [16]:
def demand(price, max_price, max_demand):
    if price > max_price:
        return 0.0
    elif price <= 0.0:
        return max_demand
    else:
        return max_demand - np.power(price, 2.0) * max_demand / np.power(max_price, 2.0)
    
def cost(amount, kwh_per_plant, cost_per_plant, max_plant):
    if amount <= 0.0:
        return 0.0
    elif amount > kwh_per_plant * max_plant:
        return np.finfo(np.float64).max
    else:
        return np.ceil(amount / kwh_per_plant) * cost_per_plant

vec_demand = np.vectorize(demand)
vec_cost = np.vectorize(cost)

def market_model(max_prices, max_demands, kwh_per_plant, cost_per_plant, max_plant):
    def fitness(x):
        #x = np.maximum(x, 0.0)
        e, s, p = np.split(x, 3)
        s = np.maximum(s, 0.0)
        d = vec_demand(p, max_prices, max_demands)
        c = vec_cost(e, kwh_per_plant, cost_per_plant, max_plant)
        revenue = np.sum(np.minimum(d, s) * p)
        total_cost = np.sum(c) + np.maximum(np.sum(s) - np.sum(e), 0.0) * purchase_cost
        profit = revenue - total_cost
        return profit
    return fitness

## Experiment

In [20]:
# model parameters
max_prices = np.array([0.45, 0.25, 0.2])
max_demands = np.array([2000000.0, 30000000.0, 20000000.0])
kwh_per_plant = np.array([50000.0, 600000.0, 4000000.0])
cost_per_plant = np.array([10000.0, 80000.0, 400000.0])
max_plant = np.array([100, 50, 3])
purchase_cost = 0.6
fitness = market_model(max_prices, max_demands, kwh_per_plant, cost_per_plant, max_plant)

# initialization bounds
max_values = np.concatenate([(kwh_per_plant * max_plant), max_demands, max_prices])
min_values = np.zeros(9)

# de parameters
F = 0.5
Cr = 0.4
N = 30

max_iterations = 50000

population = init_uniform_random(10, min_values, max_values)
for i in range(max_iterations):
    population = de(population, fitness, F, Cr)
    best_solution = max(population, key=fitness)
    best_fit = fitness(best_solution)
    if i % 500 == 0:
        print("[{:06d}] fitness: {:.2f}".format(i, best_fit))

[000000] fitness: -1016428.06
[000500] fitness: 1240969.56
[001000] fitness: 1384398.46
[001500] fitness: 1396827.50
[002000] fitness: 1403100.90
[002500] fitness: 1411321.62
[003000] fitness: 1415096.53
[003500] fitness: 1415721.73
[004000] fitness: 1421602.26
[004500] fitness: 1421602.26
[005000] fitness: 1421662.11
[005500] fitness: 1421662.11
[006000] fitness: 1421662.11
[006500] fitness: 1421677.79
[007000] fitness: 1421708.81
[007500] fitness: 1421708.81
[008000] fitness: 1422123.09
[008500] fitness: 1422142.45
[009000] fitness: 1422905.00
[009500] fitness: 1423161.98
[010000] fitness: 1423161.98
[010500] fitness: 1423198.31
[011000] fitness: 1423200.83
[011500] fitness: 1423281.65
[012000] fitness: 1423351.14
[012500] fitness: 1423353.41
[013000] fitness: 1423394.05
[013500] fitness: 1423409.05
[014000] fitness: 1423430.57
[014500] fitness: 1423430.57
[015000] fitness: 1423461.90
[015500] fitness: 1423496.80
[016000] fitness: 1423527.93
[016500] fitness: 1423585.21
[017000] fitn

KeyboardInterrupt: 